In [1]:
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf
import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
csv = pd.read_csv("cleaned_slayer.csv")

In [4]:
csv

,Unnamed: 0,section_id,content,file_name
0,0,##87151,43 ripsnorter unplayable watson is gone and h...,5
1,1,##97951,welcome to the world confederation for physic...,5
2,2,##209450,just over a week ago australia were bowled ou...,5
3,3,##239752,release title 345612toolong 2007 release date...,5
4,4,##411252,lte both radio and core network evolution is ...,5
...,...,...,...,...
1645,1645,##1151552,study could advantageously be carried on by s...,16
1646,1646,##1153651,the official bit the concept of organic expre...,16
1647,1647,##1154050,nietzsche hegel and schopenhauer hegel and th...,16
1648,1648,##1154350,the forex market the foreign exchange market ...,16


In [5]:
data_texts = csv['content'].to_list()

data_labels = csv['file_name'].to_list()

In [6]:
#Train Test Split
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.2, random_state = 0 )


train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.01, random_state = 0 )


In [7]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch

from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=-1)  # Get the index of the highest logit as the predicted class
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# Model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Custom Dataset class for PyTorch
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Convert to PyTorch Datasets
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)

# Model definition
trainer_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=7,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=1e-5,               
    logging_dir='./logs',            
    evaluation_strategy="steps",     
    eval_steps=100                   
)

# Trainer setup
trainer = Trainer(
    model=trainer_model,                 
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning:

`evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead



Step,Training Loss,Validation Loss,Accuracy
100,No log,0.881262,0.309091
200,No log,0.663727,0.430303
300,No log,0.623959,0.430303
400,No log,0.659847,0.427273
500,0.550700,0.779766,0.442424


TrainOutput(global_step=574, training_loss=0.49774121573577773, metrics={'train_runtime': 1351.6125, 'train_samples_per_second': 6.764, 'train_steps_per_second': 0.425, 'total_flos': 1211081748817920.0, 'train_loss': 0.49774121573577773, 'epoch': 7.0})

In [8]:
# Evaluate the model on the validation dataset
# Evaluate the model
eval_results = trainer.evaluate()

# Output validation loss and accuracy
validation_loss = eval_results.get("eval_loss", None)
accuracy = eval_results.get("eval_accuracy", None)

print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {accuracy}")



Validation Loss: 0.8993900418281555
Validation Accuracy: 0.4212121212121212


In [11]:
import torch
#Inference
# Tokenize the test input
test_text = test_texts
test_encoding = tokenizer(test_text, truncation=True, padding=True, max_length=512, return_tensors='pt')

# Set the model to evaluation mode
trainer_model.eval()

# Move tensors and model to the same device (CPU or GPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
trainer_model.to(device)
test_encoding = {key: val.to(device) for key, val in test_encoding.items()}

# Make predictions
with torch.no_grad():  # Disable gradient computation for inference
    output = trainer_model(**test_encoding)

# Get the predicted class index
logits = output.logits  # Access logits from the output
predicted_class = torch.argmax(logits, dim=1).item()

# Print the result
print(f"Predicted class: {predicted_class}")



Predicted class: 1
